#### Hierarchial Agent Teams
We've introduced the concept of single supervisor node to route work between different worker nodes. But what if the job for a single worker becomes too complex? What if the number of workers becomes too large? For some applications, the system may be more effective it work is distributed hierarchically. You can do this by composeing differnt subgraphs and creating a top-level supervisor, along with mid-level supervisors. To do this, let's build a simple research assistant!


In [4]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, FunctionMessage, BaseMessage
import matplotlib.pyplot as plt
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import tool
from typing import List

In [5]:
@tool
def scrape_webpages(urls: List[str]) -> str:
    """
    Use requests and bs4 to scrape the provided web pages for detailed information
    """
    loader = WebBaseLoader(urls)
    docs = loader.load()
    
    return "\n\n".join([f"<Document name='{doc.metadata.get('title', '')}'> \n {doc.page_content}" for doc in docs])

##### Document writing team tools 
Next up, we will give some tools to the doc writing team to use. We define some bare-bones file-access tools below. 
Note that this gives the agents access to your file-system, which can be unsafe. We also haven't optimized the tool description for performance

In [7]:
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Dict, Optional, TypedDict, Annotated
from langchain_experimental.utilities import PythonREPL


_TEMP_DIRECTORY = TemporaryDirectory()
WORKING_DIRECTORY = Path(_TEMP_DIRECTORY.name)

@tool
def create_outline(
    points: Annotated[List[str], "List of main points or sections"],
    file_name: Annotated[str, "File path to save the outline"]
) -> Annotated[str, "Path of the saved outline file."]:
    """Create and save an outline"""
    with (WORKING_DIRECTORY /file_name).open("w") as file:
        for i, point in enumerate(points):
            file.write(f"{i+1}. {point}\n")
    return f"Outline saved to {file_name}"

In [8]:
@tool
def read_document(
    file_name: Annotated[str, "file path to ave the document"],
    start: Annotated[Optional[int], "the start line. Default is 0"] = None, 
    end: Annotated[Optional[int], "the end line. Default is None"] = None
) -> str:
    """
    Read the specified document
    """
    with (WORKING_DIRECTORY / file_name).open("r") as file:
        lines = file.readlines()
    if start is not None:
        start = 0
    return "\n".join(lines[start:end])

@tool
def write_document(
    content: Annotated[str, "text content to be written in the document"],
    file_name: Annotated[str, "file path to save the document"],
    inserts: Annotated[Dict[int, str], "Dictionary where key is the line number (1-indexed) and value is the text to be inserted at the line"]
) -> Annotated[str, "path of the saved document file"]:
    """ 
    Create and save a text document
    """
    with (WORKING_DIRECTORY / file_name).open("w") as file:
        file.write(content)
    return f"Document saved to {file_name}"

@tool
def edit_document(
    file_name:Annotated[str, "path of the document to be edited"],
    inserts: Annotated[Dict[int, str], "dictionary where key is the line number (1-indexed) and value is the text to be inserted at the line."]
) -> Annotated[str, "path of the edited document file."]:
    """ 
    Edit a document by inserting text at specific line numbers.
    """
    with (WORKING_DIRECTORY /file_name).open("r") as file:
        lines = file.readlines()
    
    sorted_inserts = sorted(inserts.items())
    
    for line_number, text in sorted_inserts:
        if 1 <= line_number <= len(lines) + 1:
            lines.insert(line_number - 1, text + "\n")
        else:
            return f"Error: Line number {line_number} is out of range"
    
    with (WORKING_DIRECTORY/ file_name).open("w") as file:
        file.writelines(lines)
    
    return f"Document edited and daved to {file_name}"

In [9]:
repl = PythonREPL()

@tool
def python_repl(
    code: Annotated[str, "The python code to execute to generate your chart."]
):
    """
    Use this to execute python code. If you want to see the output of a value, you should print it out with `print(...)`. This is visible to the user
    """
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Succesfully executed: \n ```python\n{code}\n ```\nStdout: {result}"


##### Helper Utilities
We are going to create a few utility functions to make it more concise when we want to:
- Create a worker agent
- Create a supervisor for the sub-graph

In [11]:
from typing import Any, Callable, List, Optional, TypedDict, Union

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool
from langgraph.graph import END, StateGraph


In [12]:

def create_agent(
    llm: ChatOpenAI, 
    tools: list, 
    system_prompt: str
) ->str: 
    """
    Create a function-calling agent and add it to the graph
    """
    system_prompt += "\nWork autonomously according to your speciality, using the tools available to you. Do not ask for clarification. Your other team members (and other teams) will collaborate with you with their own specialities. You are chosen for a reason! You are one of the following team members {team_members}."
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ]
    )
    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [13]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}


def create_team_supervisor(llm: ChatOpenAI, system_prompt, members) -> str:
    """An LLM-based router"""
    options = ["FINISH"] + members
    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [
                        {"enum": options},
                    ],
                },
            },
            "required": ["next"],
        },
    }
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above, who should act next?"
                " Or should we FINISH? Select one of: {options}",
            ),
        ]
    ).partial(options=str(options), team_members=", ".join(members))
    return (
        prompt
        | llm.bind_functions(functions=[function_def], function_call="route")
        | JsonOutputFunctionsParser()
    )
    

##### Define agent teams
Now we can get to define our hierachical teams.

##### Research Team
The research team will have a search agent and a we scraping "research_agent" as the two worker nodes. Let's create those, as well as teh team supervisor

In [15]:
load_dotenv()

True

In [16]:
import functools
import operator

class ResearchTeamState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: List[str]
    next: str 
    
    
tavily_tool = TavilySearchResults(max_results=1)
    
llm = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")

search_agent = create_agent(llm, [tavily_tool],"You are a research assistant who can search for up-to-date info using teh tavily search engine")
search_node = functools.partial(agent_node, agent=search_agent, name="Search")

research_agent = create_agent(llm, [scrape_webpages], "You are a research assistant who can scrape specified urls for more detailed information using the scrape_webpages tool")
research_node = functools.partial(agent_node, agent=research_agent, name="Web Scrapper")

supervisor_agent = create_team_supervisor(
    llm, 
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  Search, Web Scraper. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.",
    ["Search", "Web Scraper"],
)

Now that we've created the necassary components, defining their interactions is easy. Add the nodes to the team graph, and define the edges, which determine the transition criteria

In [20]:
research_graph = StateGraph(ResearchTeamState)
research_graph.add_node("Search", search_node)
research_graph.add_node("Web Scrapper", research_node)
research_graph.add_node("supervisor", supervisor_agent)


# define the control flow
research_graph.add_edge("Search", "supervisor")
research_graph.add_edge("Web Scrapper", "supervisor")

research_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {"Search": "Search", "Web Scrapper": "Web Scrapper", "FINISH": END}
)

research_graph.set_entry_point("supervisor")
chain = research_graph.compile()


def enter_chain(message: str):
    results = {
        "messages": [HumanMessage(content=message)],
    }
    return results


research_chain = enter_chain | chain


In [21]:
for s in research_chain.stream(
    "when is Taylor Swift's next tour?", {"recursion_limit": 100}
):
    if "__end__" not in s:
        print(s)
        print("---")

OutputParserException: Could not parse function call: 'function_call'

##### Document writing team
Create the document writing team below using a similar approach. this time, we will give each agent access to different file writing tools. 
Note that we are giving file-system access to our agent here, which is not safe in all cases

In [23]:
import operator
from pathlib import Path


class DocWritingState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: str
    next: str
    current_files: str
    
    
    
def prelude(state):
    written_files = []
    if not WORKING_DIRECTORY.exists():
        WORKING_DIRECTORY.mkdir()
    try:
        written_files = [
            f.relative_to(WORKING_DIRECTORY) for f in WORKING_DIRECTORY.rglob("*")
        ]
    except:
        pass 
    
    if not written_files:
        return {**state, "current_files": "No files written"}
    return {
        **state, 
        "current_files": "\nBelow are files your team has written to the directory: \n" + "\n".join([f" - {f}" for f in written_files])
    }
    
llm = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")


doc_writer_agent = create_agent(
    llm, 
    [write_document, edit_document, read_document],
    "You are an expert writing a reasearch document. \n"
    "Below are files currently in your directory: \n{currently_files}"
)

context_aware_doc_writer_agent = prelude | doc_writer_agent
doc_writer_agent = functools.partial(
    agent_node, agent=context_aware_doc_writer_agent, name="Doc Writer"
)

note_taking_agent = create_agent(
    llm,
    [create_outline, read_document],
    "You are an expert senior researcher tasked with writing a paper outline and taking notes to craft a perfect paper. {current_files}",
)
context_aware_note_taking_agent = prelude | note_taking_agent
note_taking_node = functools.partial(agent_node, agent=context_aware_note_taking_agent, name="Note Taker")


chart_generating_agent = create_agent(
    llm,
    [read_document, python_repl],
    "You are a data viz expert tasked with generating charts for a research project {current_files}",
)

context_aware_chart_generating_agent = prelude | chart_generating_agent
chart_generating_node = functools.partial(agent_node, agent=context_aware_chart_generating_agent, name="Chart Generator")


doc_writing_supervisor = create_team_supervisor(
    llm, 
     "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {team_members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.",
    ["Doc Writer", "Note Taker", "Chart Generator"],
)


In [24]:
authoring_graph = StateGraph(DocWritingState)
authoring_graph.add_node("Doc Writer", doc_writer_agent)
authoring_graph.add_node("Note Taker", note_taking_agent)
authoring_graph.add_node("Chart Generator", chart_generating_agent)
authoring_graph.add_node("supervisor", doc_writing_supervisor)

# add the edges that always occur
authoring_graph.add_edge("Doc Writer", "supervisor")
authoring_graph.add_edge("Note Taker", "supervisor")
authoring_graph.add_edge("Chart Generator", "supervisor")

authoring_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "Doc Writer" : "Doc Writer",
        "Note Taker" : "Note Taker",
        "Chart Generator": "Chart Generator",
        "FINISH": END,
    }
)

authoring_graph.set_entry_point("supervisor")


def enter_chain(message: str, members: List[str]):
    results = {
        "messages": [HumanMessage(content=message)],
        "team_members": ",".join(members)
    }
    return results


authoring_chain = (
    functools.partial(enter_chain, members=authoring_graph.nodes) | authoring_graph.compile()
)

In [25]:
for s in authoring_chain.stream(
    "Write an outline for poem and then write the poem to disk.",
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("---")

OutputParserException: Could not parse function call: 'function_call'

##### Add Layers
In this design, we are enforcing a top-down planning policy. We've created two graphs already, but we have to decide how to route work betwwen the two.

We'll create a third graph to orchestrate the previous two, and add some connectors to define how this top-level state is shared between the different graphs

In [26]:
supervisor_node =  create_team_supervisor(
    llm, 
    "You are a supervisor tasked with managing a conversation between the"
    " following teams: {team_members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.",
    ["Research team", "Paper writing team"],
)

In [28]:
class State(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    next: str
    
def get_last_message(state: State) -> str:
    return state["messages"][-1].content


def join_graph(response: dict):
    return {"messages": [response["messages"][-1]]}



super_graph = StateGraph(State)

super_graph.add_node("Research team", get_last_message | research_chain | join_graph)
super_graph.add_node("Paper writing team", get_last_message | authoring_chain | join_graph)

super_graph.add_node("supervisor", supervisor_node)


super_graph.add_edge("Research team", "supervisor")
super_graph.add_edge("Paper writing team", "supervisor")

super_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "Paper writing team": "Paper writing team",
        "Research team": "Research team",
        "FINISH": END,
    }
)

super_graph.set_entry_point("supervisor")
super_graph = super_graph.compile()




In [29]:
for s in super_graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Write a brief research report on the North American sturgeon. Include a chart."
            )
        ],
    },
    {"recursion_limit": 150},
):
    if "__end__" not in s:
        print(s)
        print("---")

OutputParserException: Could not parse function call: 'function_call'